In [1]:
from typing import Any, Mapping, Iterable, Iterator, Tuple

from pytrustfall import (
    Adapter, DataContext, FrontendError, 
    ParseError, Schema, QueryArgumentsError,
    ValidationError, execute_query,
)

In [2]:
number_names = [
    "zero", "one", "two", "three", "four", 
    "five", "six", "seven", "eight", "nine", "ten",
]

Token = int

class NumbersAdapter(Adapter[Token]):
    def get_starting_tokens(
        self,
        edge_name: str,
        parameters: Mapping[str, Any],
        *args: Any,
        **kwargs: Any,
    ) -> Iterable[Token]:
        print("get_starting_tokens call with input:", edge_name, parameters)
        try:
            max_value = parameters["max"]
            yield from range(0, max_value)
        except Exception as e:
            print("get_starting_tokens error", e)
            raise
    
    def project_property(
        self,
        data_contexts: Iterator[DataContext[Token]],
        type_name: str,
        field_name: str,
        *args: Any,
        **kwargs: Any,
    ) -> Iterable[Tuple[DataContext[Token], Any]]:
        print("project_property call with input:", type_name, field_name)
        try:
            for context in data_contexts:
                token = context.current_token
                value = None
                if token is not None:
                    if field_name == "value":
                        value = token
                    elif field_name == "name":
                        value = number_names[token]
                    else:
                        raise NotImplementedError()

                yield (context, value)
        except Exception as e:
            print("project_property error", e)
            raise   
    
    def project_neighbors(
        self,
        data_contexts: Iterator[DataContext[Token]],
        type_name: str,
        edge_name: str,
        parameters: Mapping[str, Any],
        *args: Any,
        **kwargs: Any,
    ) -> Iterable[Tuple[DataContext[Token], Iterable[Token]]]:
        print("project_property call with input:", type_name, edge_name, parameters)
        try:
            for context in data_contexts:
                token = context.current_token
                neighbors = []
                if token is not None:
                    if edge_name == "multiple":
                        if token > 0:
                            max_value = parameters["max"]
                            neighbors = range(2 * token, max_value * token + 1, token)
                    elif edge_name == "predecessor":
                        if token > 0:
                            neighbors = [token - 1]
                    elif edge_name == "successor":
                        neighbors = [token + 1]
                    else:
                        raise NotImplementedError()

                yield (context, neighbors)
        except Exception as e:
            print("project_property error", e)
            raise
    
    def can_coerce_to_type(
        self,
        data_contexts: Iterator[DataContext[Token]],
        type_name: str,
        coerce_to_type: str,
        *args: Any,
        **kwargs: Any,
    ) -> Iterable[Tuple[DataContext[Token], bool]]:
        raise NotImplementedError()


In [3]:
adapter = NumbersAdapter()
with open("./numbers.graphql") as f:
    schema = Schema(f.read())

def execute(query, args):
    return list(execute_query(adapter, schema, query, args))

In [4]:
query = """
{
    Number(max: 10) {
        value @output
        name @output
    }
}
"""
args = {}
execute(query, args)

project_property call with input: Number value
project_property call with input: Number name
get_starting_tokens call with input: Number {'max': 10}


[{'value': 0, 'name': 'zero'},
 {'value': 1, 'name': 'one'},
 {'value': 2, 'name': 'two'},
 {'name': 'three', 'value': 3},
 {'value': 4, 'name': 'four'},
 {'value': 5, 'name': 'five'},
 {'value': 6, 'name': 'six'},
 {'value': 7, 'name': 'seven'},
 {'name': 'eight', 'value': 8},
 {'name': 'nine', 'value': 9}]

In [5]:
query = """
{
    Number(max: 4) {
        value @output
        
        multiple(max: 3) {
            mul: value @output
        }
    }
}
"""
args = {}
execute(query, args)

project_property call with input: Number value
project_property call with input: Number value
project_property call with input: Number multiple {'max': 3}
get_starting_tokens call with input: Number {'max': 4}


[{'mul': 2, 'value': 1},
 {'mul': 3, 'value': 1},
 {'mul': 4, 'value': 2},
 {'value': 2, 'mul': 6},
 {'value': 3, 'mul': 6},
 {'mul': 9, 'value': 3}]

In [6]:
query = """
{
    Number(max: 4) {
        value @output @filter(op: ">", value: ["$bound"])
    }
}
"""
args = {
    "bound": 2,
}
execute(query, args)

project_property call with input: Number value
project_property call with input: Number value
get_starting_tokens call with input: Number {'max': 4}


[{'value': 3}]

In [7]:
query = "this isn't GraphQL"
args = {}
try:
    execute(query, args)
    assert(False)
except ParseError as e:
    print(e)

Input is not valid GraphQL:  --> 1:1
  |
1 | this isn't GraphQL
  | ^---
  |
  = expected executable_definition


In [8]:
query = """
{
    Number(max: 4) {
        nonexistent @output
    }
}
"""
args = {}
try:
    execute(query, args)
    assert(False)
except ValidationError as e:
    print(e)

The referenced path does not exist in the schema: ["Number", "nonexistent"]


In [9]:
query = """
{
    Number(max: 4) {
        value @output
        value @output
    }
}
"""
args = {}
try:
    execute(query, args)
    assert(False)
except FrontendError as e:
    print(e)

Multiple fields are being output under the same name: DuplicatedNamesConflict { duplicates: {"value": [("Number", "value"), ("Number", "value")]} }


In [10]:
query = """
{
    Number(max: 4) {
        value @output @filter(op: ">", value: ["$bound"])
    }
}
"""
args = {
    "not_bound": "123",
}
try:
    execute(query, args)
    assert(False)
except QueryArgumentsError as e:
    print(e)

Multiple argument errors: [
  One or more arguments required by this query were not provided: ["bound"];
  One or more of the provided arguments are not used in this query: ["not_bound"];
]
